# Listen to the 'Game of Thrones'-Hashtags
Listens and stores the twitter.

In [1]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

import dateutil
import time
import json
import csv
import os
import re

In [2]:
with open('Twitter_API_Auth.csv', 'r') as f:
    auth = [line.strip() for line in f]
    
ckey = auth[0]
csecret = auth[1]
atoken = auth[2]
asecret = auth[3]

In [3]:
path = 'GoT_finale.csv'

keys = ['created_at', 'id', 'text', 'source', 'lang', 'timestamp_ms', 'user_id', 'user_name', 'user_status', 'user_follower', 'user_friends', 'user_rate', 'user_created']

if not os.path.isfile(path):
    with open(path, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(keys)

In [ ]:
class listener(StreamListener):
    def on_data(self, data):
        try:
            loaded = json.loads(data)
            
            tweet = []
            tweet.append(dateutil.parser.parse(loaded[keys[0]]))
            
            for key in keys[1:6]:
                tweet.append(loaded[key])
            user = loaded[keys[6][:4]]            
            
            id = user['id']
            name = user['name']
            sta = user['statuses_count']
            fol = user['followers_count']
            fri = user['friends_count']
            rate = fol/fri if fri!=0 else 9000
            try:
                created = dateutil.parser.parse(user['created_at'])
            except:
                created = None
            tweet.extend([id, name, sta, fol, fri, rate, created])
            
            with open(path, 'a') as f:
                writer = csv.writer(f)
                writer.writerow(tweet)
        except:
            pass
        #except BaseException as e:
            #print('Failed ondata, ', str(e))
            #time.sleep(5)
        return True
    
    def on_error(self, status):
        print(status)
        
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

hashtags = ['#gots{}e{}'.format(s, e) for s in range(1, 8) for e in range(1, 11)]
hashtags = ['#gameofthones', '#gameofthrones', '#gameofthrones7', '#gameofthronesfinale', '#gameofthroness7', '#gamesofthrones', '#got', '#got7', '#gotfinale', '#gots7', '#gots7e7', '#juegodetronos']
while True:
    try:
        twitterStream = Stream(auth, listener())
        twitterStream.filter(track=hashtags)
        #twitterStream.filter(track=['the', 'to', 'a', 'of', 'in'])
    except:
        print('Fehler')
        pass